# Optunaで最適なハイパーパラメータを探索する
- データセット：`BIAS-PROFS`
- 使用したモデル：`google/vit-base-patch16-224-in21k`

## 1. 必要なライブラリのインストール

In [1]:
import os
import evaluate
import numpy as np
import optuna
import pandas as pd
import multiprocessing
import matplotlib.pyplot as plt

from PIL import Image
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, ViTImageProcessorFast, ViTForImageClassification, TrainingArguments, Trainer, TrainerCallback
from optuna.integration import PyTorchLightningPruningCallback
from optuna.exceptions import TrialPruned

from torch.utils.data import Dataset, Subset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay

label2id = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4}
id2label = {v: k for k, v in label2id.items()}

MODEL_NAME = "google/vit-base-patch16-224-in21k"

## 2. Optuna用の `TrainerCallback`

In [2]:
class OptunaPruningCallback(TrainerCallback):
    def __init__(self, trial, metric_name="eval_f1"):
        self.trial = trial
        self.metric_name = metric_name

    # on_evaluate が呼ばれるたびに metrics を受け取り report する
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is None:
            return
        # ここは Trainer の compute_metrics が返す key と合わせる
        val = metrics.get(self.metric_name)
        if val is None:
            return
        step = int(state.epoch) if state.epoch is not None else 0
        self.trial.report(val, step=step)
        if self.trial.should_prune():
            raise TrialPruned()

## 3. 画像とラベルの対応を取る `Dataset` を作成

In [3]:
class GPCRImageDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path, header=None, names=["num", "label", "accession"])
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['num']}.png")
        image = Image.open(img_path).convert("RGB")
        label = label2id[row["label"]]

        if self.transform:
            image = self.transform(image)
        return {"pixel_values": image, "label": label}

## 4. 画像前処理（ViT用）

In [4]:
processor = ViTImageProcessorFast.from_pretrained(MODEL_NAME)
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=processor.image_mean, std=processor.image_std)
])

## 5. `Trainer` を作成

In [5]:
accuracy = evaluate.load("accuracy")
dataset = GPCRImageDataset("../gpcr_labels.csv", "../graphs/gradation", transform=transform)

# StratifiedKFoldはラベル情報を使う
labels = dataset.data["label"].values
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## 6. 目的関数

In [14]:
def objective(trial):
    # --- ハイパーパラメータ空間 ---
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 5e-4)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    wd = trial.suggest_loguniform("weight_decay", 1e-6, 1e-2)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)

    fold_f1_scores = []  # 5-fold の結果を格納

    # --- 再現性（任意） ---
    seed = 42

    for fold, (train_idx, val_idx) in enumerate(kfold.split(np.zeros(len(labels)), labels)):
        print(f"\033[34m[Fold {fold+1} / 5]\033[0m")
        
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

        # --- モデル & processor の準備
        processor = ViTImageProcessorFast.from_pretrained(MODEL_NAME)
    
        model = ViTForImageClassification.from_pretrained(
            MODEL_NAME,
            num_labels=5,
            id2label=id2label,
            label2id=label2id
        )
    
        # set dropout if supported
        try:
            model.config.classifier_dropout = dropout
        except Exception:
            pass

        # --- TrainingArguments (短めの試行にする) ---
        output_dir = f"./optuna_tmp/trial_{trial.number}"
        
        args = TrainingArguments(
            output_dir=output_dir,
            learning_rate=lr,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=wd,
            num_train_epochs=5,
            eval_strategy="epoch",
            save_strategy="no",
            logging_strategy="epoch",
            load_best_model_at_end=False,
            report_to="none",
            dataloader_num_workers=8,
            fp16=True
        )

        # compute_metrics は既存の関数を流用
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            preds = np.argmax(logits, axis=-1)
            acc = accuracy.compute(predictions=preds, references=labels)["accuracy"]
            f1 = f1_score(labels, preds, average="macro")
            return {"accuracy": acc, "f1": f1, "eval_f1": f1}  # eval_f1 をキーにしておく

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            tokenizer=processor
        )

        # --- 学習 ---
        try:
            trainer.train()
        except TrialPruned:
            # Optuna がプルーニングしたら trial を中断
            raise

        metrics = trainer.evaluate()
        fold_f1 = metrics["eval_f1"]
        fold_f1_scores.append(fold_f1)

        # --- プルーニング判定（fold平均で pruning） ---
        trial.report(np.mean(fold_f1_scores), fold)
        
        if trial.should_prune():
            raise optuna.TrialPruned()

    # --- 5 fold の平均スコアを返す ---
    return np.mean(fold_f1_scores)

## 7. Optuna 実行

In [16]:
study = optuna.create_study(direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)
)

# 複数試行
study.optimize(objective, n_trials=40, n_jobs=1)  # n_jobs>1 で並列実行

print("Best trial:")
trial = study.best_trial
print(trial.params)

[I 2025-11-16 15:58:22,901] A new study created in memory with name: no-name-2d47a2dd-96e7-439e-b424-63364e5a4b76
/tmp/ipykernel_331201/1193334964.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 5e-4)
/tmp/ipykernel_331201/1193334964.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  wd = trial.suggest_loguniform("weight_decay", 1e-6, 1e-2)


[Fold 1 / 5]


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_331201/1193334964.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.912400,0.678805,0.272772,0.771373
2,0.594000,0.520579,0.357152,0.882772
3,0.480700,0.448335,0.372027,0.887306
4,0.424900,0.408248,0.472898,0.914508
5,0.400300,0.398311,0.487269,0.918394


[Fold 2 / 5]


...

[Fold 5 / 5]


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_331201/1193334964.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.366300,0.192501,0.541201,0.952690
2,0.137800,0.146381,0.554978,0.962411
3,0.087900,0.165424,0.551848,0.958522
4,0.054900,0.142255,0.643577,0.966948
5,0.037600,0.142099,0.739668,0.970836


[I 2025-11-17 02:51:35,477] Trial 39 finished with value: 0.7325696194965647 and parameters: {'learning_rate': 2.1952798443320316e-05, 'batch_size': 8, 'weight_decay': 0.009878624585675323, 'dropout': 0.35130222963190855}. Best is trial 15 with value: 0.7798348328049844.


Best trial:
{'learning_rate': 6.08765385007137e-05, 'batch_size': 16, 'weight_decay': 7.018621340063472e-06, 'dropout': 0.07198371055492106}
